In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003987550735473633
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 23.683619642363464
200000 36.3865449382438
300000 39.678159641402374
400000 38.77784534056252
500000 37.25683161680017
600000 37.9267018237973
700000 38.18938568373591
800000 38.74170966359481
900000 28.846670325304906
1000000 19.790761125289507
1100000 37.1331948750069
1200000 30.25917962013389
1300000 35.01696091800754
1400000 31.819575340427853
1500000 33.44968746943361
1600000 30.342320750287854
1700000 33.48774417427844
1800000 28.52564370374655
1900000 13.894721897498417
2000000 31.978548992620933
2100000 29.09509637557649
2200000 28.89036064079
2300000 32.01725972440969
2400000 28.637663116586907
2500000 30.579186271169682
2600000 28.36147347995788
2700000 29.420854103532992
2800000 14.01425052066014
2900000 28.277831655412484
3000000 29.9885129314

29500000 30.332026941721892
29600000 28.98437920151096
29700000 28.53996494464099
29800000 27.31247163516058
29900000 26.645811997799594
30000000 11.878213751619475
30100000 26.917164242020693
30200000 26.8646369080636
30300000 28.16537185746942
30400000 28.611794436333835
30500000 30.041531290060462
30600000 28.973587270857113
30700000 15.175068386296605
30800000 29.566513747883466
30900000 29.38794491117949
31000000 29.384280381561187
31100000 28.175347800037414
31200000 14.798886558787363
31300000 29.49838328624847
31400000 28.26026917333102
31500000 28.523772374841332
31600000 26.54837872882406
31700000 24.961104189040555
31800000 15.196743623237374
31900000 22.407931167337
32000000 24.803041203880873
32100000 27.041306215277338
32200000 27.080048670632085
32300000 28.076088046608955
32400000 29.28045800251103
32500000 14.912685597049283
32600000 28.3478989037457
32700000 29.006873469354538
32800000 29.177201387805756
32900000 29.730880242188206
33000000 13.571793970776552
33100000

59300000 11.963388099720627
59400000 14.825945886119285
59500000 8.66131165085058
59600000 12.848814415383053
59700000 16.467575051378056
59800000 12.780627710525371
59900000 14.629131491939285
60000000 14.567923123626079
60100000 12.491675596705171
60200000 13.55543311320868
60300000 14.044173038444924
60400000 10.994139115924407
60500000 7.5098469625910615
60600000 12.157362944573105
60700000 17.292115724260263
60800000 16.570604861235985
60900000 9.254402167828808
61000000 16.69047320007029
61100000 16.781602341377017
61200000 14.050077033689224
61300000 6.455576594101733
61400000 12.351016161970655
61500000 16.42881185358261
61600000 19.70730331040414
61700000 19.807710908187637
61800000 20.31322936668552
61900000 13.687423054535513
62000000 13.953505072741926
62100000 14.142352699045098
62200000 19.1610756590987
62300000 5.743220561559031
62400000 12.663408589476198
62500000 20.425272113130603
62600000 11.66650298591416
62700000 4.951777742154245
62800000 13.915521411306852
629000

89000000 25.936415856794625
89100000 26.505519132417458
89200000 24.520883290589275
89300000 22.05477355504181
89400000 22.781701477700487
89500000 23.638388588579176
89600000 22.78115705547933
89700000 19.6263305882196
89800000 15.610620566378751
89900000 14.550896239949196
90000000 18.622992938187068
90100000 13.063902287413308
90200000 22.45429364269069
90300000 25.041620554650557
90400000 23.19457777653962
90500000 22.83735220975678
90600000 23.506353178448045
90700000 24.35735045403962
90800000 27.500684648209155
90900000 26.60853246939413
91000000 25.577154575909255
91100000 24.008147682486992
91200000 23.35205090101994
91300000 23.503755649830666
91400000 22.80097158164984
91500000 23.555304413538067
91600000 11.525267081975233
91700000 17.852428651075662
91800000 15.346098982050764
91900000 13.115195553389825
92000000 22.811075897705724
92100000 23.587094965580878
92200000 23.519085013314985
92300000 24.471318661660362
92400000 23.664803995225917
92500000 25.032500234324807
926

118100000 28.951265553116517
118200000 25.154926320743048
118300000 25.314589809447607
118400000 24.071392815661977
118500000 24.030625885253237
118600000 24.387215550169614
118700000 23.414702029731124
118800000 24.370390771036227
118900000 15.14058346712109
119000000 3.7405620263251462
119100000 15.88662789391464
119200000 24.39010094128806
119300000 23.014021239376675
119400000 24.85938609437458
119500000 24.15158139438771
119600000 24.573686483799843
119700000 25.165975209924024
119800000 25.075118961111325
119900000 25.152481993385443
120000000 21.807720683392574
120100000 22.414410568044936
120200000 19.86960367470141
120300000 21.44478614498836
120400000 21.468903535515267
120500000 19.84747455025649
120600000 20.861187863034527
120700000 13.862939308281156
120800000 3.744428413019051
120900000 10.445409679292373
121000000 19.163160345490837
121100000 21.01537545068002
121200000 20.452050298369745
121300000 21.52003107907817
121400000 20.563209237275004
121500000 21.038467938279

...

320500000 2.192009374081426
320600000 2.5891890532324418
320700000 5.093839144059837
320800000 5.135799194736689
320900000 5.749291155230363
321000000 5.5371693482798205
321100000 5.8542150488736135
321200000 5.825700153899372
321300000 5.18426173080839
321400000 5.6451992878715656
321500000 5.05040263361752
321600000 2.8850867018425332
321700000 4.429174864393757
321800000 5.12968581721594
321900000 4.993008336855889
322000000 5.157040351309779
322100000 4.710912698344618
322200000 4.663089348839154
322300000 2.959558747362997
322400000 1.7385491920955678
322500000 5.153396840800984
322600000 5.4869743719208275
322700000 5.8911400692888805
322800000 5.237668131774519
322900000 5.569293967081583
323000000 5.610915740026653
323100000 5.148870020324485
323200000 5.858175659667031
323300000 5.077795115351488
323400000 2.5673026724338563
323500000 4.510786364160121
323600000 4.876130741001239
323700000 4.746133988503228
323800000 5.140470150884843
323900000 4.796436401505202
324000000 4.77

348700000 3.7826699604316913
348800000 4.029805275994659
348900000 3.0950985143649343
349000000 3.5535218696144923
349100000 2.8193919638434646
349200000 2.634445686162174
349300000 3.666743426716205
349400000 2.6356856925779084
349500000 3.803427952139497
349600000 3.952293954759319
349700000 3.9248112762064333
349800000 3.0481154376255173
349900000 3.7747714962980314
350000000 3.0600066935419146
350100000 2.4903851064543456
350200000 3.2841555164918206
350300000 2.566026865866353
350400000 3.8132952751132776
350500000 3.7386407277864317
350600000 3.905450307477385
350700000 2.837669000124468
350800000 3.616724977003001
350900000 3.308620504524469
351000000 2.2439250331564446
351100000 3.6577980282390223
351200000 2.8080624120302895
351300000 3.5724875715745887
351400000 4.045284660642295
351500000 3.6047147827901638
351600000 3.5749554110490247
351700000 3.2699428567896254
351800000 3.6467171974346337
351900000 2.0347061550126675
352000000 3.388386755773653
352100000 2.92895701048862

377400000 2.1885347729957862
377500000 1.7494131003780797
377600000 2.2688028015832007
377700000 2.351919176574452
377800000 2.1426931467992993
377900000 2.1748564220167697
378000000 1.5896624435000022
378100000 2.118762776271936
378200000 1.1994290077960095
378300000 2.1520006810147994
378400000 1.6376781059345935
378500000 2.228403194803586
378600000 2.3021648068430522
378700000 2.4073180470183595
378800000 1.6136415973951221
378900000 1.3513193290937908
379000000 2.050877536648472
379100000 1.2842544855183662
379200000 1.792474956416885
379300000 1.4774799266241792
379400000 1.5764476627844572
379500000 1.6159558046266005
379600000 2.1741222836907803
379700000 1.6630008643950822
379800000 1.3310878223032092
379900000 1.6882790827161307
380000000 1.028064481482158
380100000 1.5478782313703234
380200000 1.1783392038597695
380300000 1.7696625105827253
380400000 1.5740642243270675
380500000 2.2473728081499638
380600000 1.667369401191443
380700000 1.3063425453005069
380800000 1.522731000

404600000 0.8828567446864843
404700000 1.0858128291286462
404800000 0.5974342127564848
404900000 0.3673290686200262
405000000 0.6036448790557417
405100000 0.6610936571925614
405200000 0.8347409959350527
405300000 0.2955808042522239
405400000 0.8541834807184603
405500000 1.1192401065755089
405600000 0.8075363315680126
405700000 0.6065119349391792
405800000 0.3169459917505085
405900000 0.5517535605403682
406000000 0.5933239815727969
406100000 0.5418364305561483
406200000 0.2988272873069313
406300000 0.5182694353484975
406400000 0.5779874177030722
406500000 0.8860236933909854
406600000 0.5400868859849207
406700000 0.33291506715604663
406800000 0.9142897826770411
406900000 0.3860019016203748
407000000 0.13001032682629426
407100000 0.1934314328639474
407200000 0.120501627231482
407300000 0.1320124148279329
407400000 0.22860235293726128
407500000 0.16206330610790848
407600000 0.20633695656388998
407700000 0.1932954376701249
407800000 0.12136031002263559
407900000 0.12095785092499853
40800000

430800000 0.165845687023994
430900000 0.10605716940694385
431000000 0.1460017470907867
431100000 0.06939617600415814
431200000 0.10576924176469114
431300000 0.08393991028565168
431400000 0.08706285729764236
431500000 0.08597722819858789
431600000 0.1100529212021033
431700000 0.15865333644118243
431800000 0.07150315479875273
431900000 0.0902641500325501
432000000 0.0635194583393832
432100000 0.09153514661170707
432200000 0.06465618641549349
432300000 0.07595366916158133
432400000 0.10480083597850798
432500000 0.07458405100747943
432600000 0.07247205798173613
432700000 0.05492745547006859
432800000 0.11699461650729179
432900000 0.06584653616932035
433000000 0.09773040877802174
433100000 0.05587655835399032
433200000 0.05695311367593209
433300000 0.05173111703427633
433400000 0.08636791757114728
433500000 0.06373321347542604
433600000 0.04368001664058367
433700000 0.04493912465035915
433800000 0.03953825903724299
433900000 0.044163889822857245
434000000 0.03627660066382091
434100000 0.039